In [1]:
pip install requests pandas


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
national_parks_list = [
"Acadia National Park",
"Arches National Park",
"Badlands National Park",
"Big Bend National Park",
"Biscayne National Park",
"Black Canyon Of The Gunnison National Park",
"Bryce Canyon National Park",
"Canyonlands National Park",
"Capitol Reef National Park",
"Carlsbad Caverns National Park",
"Channel Islands National Park",
"Congaree National Park",
"Crater Lake National Park",
"Cuyahoga Valley National Park",
"Death Valley National Park",
"Denali National Park & Preserve",
"Dry Tortugas National Park",
"Everglades National Park",
"Gates Of The Arctic National Park & Preserve",
"Gateway Arch National Park",
"Glacier Bay National Park & Preserve",
"Glacier National Park",
"Grand Canyon National Park",
"Grand Teton National Park",
"Great Basin National Park",
"Great Sand Dunes National Park & Preserve",
"Great Smoky Mountains National Park",
"Guadalupe Mountains National Park",
"Haleakalā National Park",
"Hawaiʻi Volcanoes National Park",
"Hot Springs National Park",
"Indiana Dunes National Park",
"Isle Royale National Park",
"Joshua Tree National Park",
"Katmai National Park & Preserve",
"Kenai Fjords National Park",
"Kobuk Valley National Park",
"Lake Clark National Park & Preserve",
"Lassen Volcanic National Park",
"Mammoth Cave National Park",
"Mesa Verde National Park",
"Mount Rainier National Park",
"National Park of American Samoa",
"New River Gorge National Park & Preserve",
"North Cascades National Park",
"Olympic National Park",
"Petrified Forest National Park",
"Pinnacles National Park",
"Redwood National and State Parks",
"Rocky Mountain National Park",
"Saguaro National Park",
"Sequoia & Kings Canyon National Parks",
"Shenandoah National Park",
"Theodore Roosevelt National Park",
"Virgin Islands National Park",
"Voyageurs National Park",
"White Sands National Park",
"Wind Cave National Park",
"Wrangell - St Elias National Park & Preserve",
"Yellowstone National Park",
"Yosemite National Park",
"Zion National Park"
]
# Sequoia & Kings Canyon National Parks refers to two adjacent national parks located in California. 
# They were originally established separately – Sequoia National Park on September 25, 1940, and Kings Canyon National Park on March 4, 1940. 
#However, they were combined into a single administrative unit on October 1, 1965.

In [2]:
from getpass import getpass

NPS_API_KEY = getpass(prompt="Enter your NPS API key: ")


Enter your NPS API key:  ········


In [156]:
import requests
import json
BASE_URL = 'https://developer.nps.gov/api/v1'

def get_all_parks(api_key):
    url = f'{BASE_URL}/parks'
    parks = []
    start = 0
    limit = 50  # Adjust the limit as needed

    while True:
        params = {
            'api_key': api_key,
            'limit': limit,
            'start': start
        }
        response = requests.get(url, params=params)
        data = response.json()
        
        # Add the data to the list
        parks.extend(data['data'])

        # Check if we have received all the records
        if len(data['data']) < limit:
            break
        
        # Move to the next page
        start += limit


    return parks

all_parks_data = get_all_parks(NPS_API_KEY)
with open('all_parks_data.json', 'w') as json_file:
    json.dump(all_parks_data, json_file, indent=4)

print("Data saved to all_parks_data.json")

Data saved to all_parks_data.json


In [157]:
len(all_parks_data)

472

In [158]:
all_parks_names = [park['fullName'] for park in all_parks_data]

In [159]:
len(national_parks_list)

62

In [160]:
def filter_national_parks(parks, park_names):
    return [park for park in parks if park['fullName'] in park_names]

national_parks_data = filter_national_parks(all_parks_data, national_parks_list)


In [161]:
len(national_parks_data)

62

In [162]:
national_parks_data
with open('raw_national_parks_data.json', 'w') as json_file:
    json.dump(national_parks_data, json_file, indent=4)

print("Data saved to raw_national_parks_data.json")

Data saved to raw_national_parks_data.json


In [2]:
import json

In [36]:
with open('raw_national_parks_data.json', 'r') as json_file:
    national_parks_data = json.load(json_file)

In [37]:
national_parks_names = [park['fullName'] for park in national_parks_data]

In [38]:
# To check any national park is missing
set_original_parks = set(national_parks_list)
set_filtered_parks = set(national_parks_names)

# Find missing parks
missing_parks = set_original_parks - set_filtered_parks

# Display missing parks
print("Missing parks from the filtered list:")
for park in missing_parks:
    print(park)

Missing parks from the filtered list:


In [39]:
# no need of multimedia, name, designation, fees
def remove_unwanted_fields(parks):
    fields_to_remove = ["multimedia", "name", "designation", "fees"]
    for park in parks:
        for field in fields_to_remove:
            if field in park:
                del park[field]
    return parks

# Remove the fields and print the result
cleaned_parks_data = remove_unwanted_fields(national_parks_data)


In [40]:
##converted activities and topics into a list
def process_parks_data(parks):
    for park in parks:
        # Safeguard: Ensure 'activities' is a list of dictionaries
        if 'activities' in park and isinstance(park['activities'], list):
            park['activities'] = [
                activity['name'] for activity in park['activities'] 
                if isinstance(activity, dict) and 'name' in activity
            ]
        
        # Safeguard: Ensure 'topics' is a list of dictionaries
        if 'topics' in park and isinstance(park['topics'], list):
            park['topics'] = [
                topic['name'] for topic in park['topics'] 
                if isinstance(topic, dict) and 'name' in topic
            ]
    
    return parks

# Example usage with cleaned_parks_data
processed_parks_data = process_parks_data(cleaned_parks_data)


In [41]:
#processed the contacts 
def process_contacts(parks):
    for park in parks:
        # Initialize default values
        phone_number = None
        email_address = None
        
        # Check for 'contacts' key in the park dictionary
        if 'contacts' in park:
            contacts = park['contacts']
            
            # Extract phone number with type 'Voice'
            if 'phoneNumbers' in contacts:
                for phone in contacts['phoneNumbers']:
                    if phone.get('type') == 'Voice':
                        phone_number = phone.get('phoneNumber')
                        break
            
            # Extract email address
            if 'emailAddresses' in contacts:
                email_address = contacts['emailAddresses'][0].get('emailAddress') if contacts['emailAddresses'] else None
            
            # Update the 'contacts' field
            park['contacts'] = {
                'phoneNumber': phone_number,
                'emailAddress': email_address
            }
    
    return parks

processed_parks_data = process_contacts(processed_parks_data)


In [42]:
def processing_address_hours(parks):
    for park in parks:
        # Remove 'exceptions' from 'operatingHours'
        if 'operatingHours' in park:
            for hours in park['operatingHours']:
                if 'exceptions' in hours:
                    del hours['exceptions']
        
        # Remove the mailing address from 'addresses'
        if 'addresses' in park:
            park['addresses'] = [address for address in park['addresses'] if address.get('type') != 'Mailing']
        
        # Remove 'relevanceScore'
        if 'relevanceScore' in park:
            del park['relevanceScore']
    
    return parks

processed_parks_data = processing_address_hours(processed_parks_data)


In [43]:
def simplify_images(parks):
    for park in parks:
        # Simplify the 'images' field
        if 'images' in park:
            park['images'] = [{'title': image.get('title'), 'url': image.get('url')} for image in park['images']]
    
    return parks

simplified_parks_data = simplify_images(processed_parks_data)


In [44]:
def remove_unwanted_fields(parks):
    fields_to_remove = ["latitude", "longitude", "latLong", "entranceFees", "entrancePasses"]
    for park in parks:
        for field in fields_to_remove:
            if field in park:
                del park[field]
        
    return parks

simplified_parks_data = remove_unwanted_fields(simplified_parks_data)


In [46]:
def format_address(address):
    # Extract non-empty values from the address dictionary
    parts = [address.get('line1', ''), 
             address.get('line2', ''), 
             address.get('line3', ''), 
             address.get('city', ''), 
             address.get('stateCode', ''), 
             address.get('countryCode', '')]

    # Filter out empty strings and join the remaining parts with commas
    formatted_address = ', '.join([part for part in parts if part])

    return formatted_address

def formating_addresses(parks):    
    for park in parks:
        # Format addresses if present
        if 'addresses' in park and isinstance(park['addresses'], list):
            # Combine all formatted addresses into a single string
            formatted_addresses = ', '.join([format_address(address) for address in park['addresses']])
            # Set addresses as a single string
            park['addresses'] = formatted_addresses
    
    return parks

# Call the function on your dataset
simplified_parks_data = formating_addresses(simplified_parks_data)


In [50]:
# Check the first record to verify the result
simplified_parks_data[5]

{'id': 'BDBD573F-97EF-44E7-A579-471679F2C42A',
 'url': 'https://www.nps.gov/blca/index.htm',
 'fullName': 'Black Canyon Of The Gunnison National Park',
 'parkCode': 'blca',
 'description': 'Big enough to be overwhelming, still intimate enough to feel the pulse of time, Black Canyon of the Gunnison National Park exposes you to some of the steepest cliffs, oldest rock, and craggiest spires in North America. With two million years to work, the Gunnison River, along with the forces of weathering, has sculpted this vertical wilderness of rock, water, and sky.',
 'activities': ['Camping',
  'Climbing',
  'Fishing',
  'Hiking',
  'Backcountry Hiking',
  'Horse Trekking',
  'Horseback Riding',
  'Paddling',
  'Kayaking',
  'Skiing',
  'Cross-Country Skiing',
  'Snowshoeing',
  'Wildlife Watching'],
 'topics': ['Explorers and Expeditions',
  'Transportation',
  'Trains and Railroads',
  'Animals',
  'Birds',
  'Canyons and Canyonlands',
  'Geology'],
 'states': 'CO',
 'contacts': {'phoneNumber'

In [53]:
def remove_operating_hours(parks):    
    for park in parks:
        # Remove operatingHours field if present
        if 'operatingHours' in park:
            del park['operatingHours']
    
    return parks

# Apply the function
simplified_parks_data = remove_operating_hours(simplified_parks_data)

In [54]:
def simplify_contacts(parks):    
    for park in parks:
        # Simplify contacts field if present
        if 'contacts' in park:
            phone = park['contacts'].get('phoneNumber', '')
            email = park['contacts'].get('emailAddress', '')
            
            # Create a simplified string for contacts
            simplified_contact = f"{phone}, {email}" if phone and email else phone or email
            park['contacts'] = simplified_contact
    
    return parks

# Apply the function
simplified_parks_data = simplify_contacts(simplified_parks_data)


In [57]:
# Output the result
simplified_parks_data[0]

{'id': '6DA17C86-088E-4B4D-B862-7C1BD5CF236B',
 'url': 'https://www.nps.gov/acad/index.htm',
 'fullName': 'Acadia National Park',
 'parkCode': 'acad',
 'description': "Acadia National Park protects the natural beauty of the highest rocky headlands along the Atlantic coastline of the United States, an abundance of habitats, and a rich cultural heritage. At 4 million visits a year, it's one of the top 10 most-visited national parks in the United States. Visitors enjoy 27 miles of historic motor roads, 158 miles of hiking trails, and 45 miles of carriage roads.",
 'activities': ['Arts and Culture',
  'Cultural Demonstrations',
  'Astronomy',
  'Stargazing',
  'Biking',
  'Boating',
  'Camping',
  'Group Camping',
  'Climbing',
  'Rock Climbing',
  'Compass and GPS',
  'Geocaching',
  'Fishing',
  'Freshwater Fishing',
  'Fly Fishing',
  'Saltwater Fishing',
  'Food',
  'Picnicking',
  'Guided Tours',
  'Bus/Shuttle Guided Tour',
  'Boat Tour',
  'Hands-On',
  'Citizen Science',
  'Hiking'

In [58]:
with open('simplified_national_parks_data.json', 'w') as json_file:
    json.dump(simplified_parks_data, json_file, indent=4)

print("Data saved to simplified_national_parks_data.json")

Data saved to simplified_national_parks_data.json
